<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EC%9E%90%EC%A0%84%EA%B1%B0%EB%A0%8C%ED%83%88%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A1_to_datetime_RadomForestRegressor_RMSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 107kB/s]


In [2]:
!kaggle competitions download -c bike-sharing-demand

bike-sharing-demand.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip /content/bike-sharing-demand.zip

Archive:  /content/bike-sharing-demand.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    
    y_train = X_train[['id', 'casual', 'registered', 'count']]
    X_train = X_train.drop(columns=target)

    
    y_test = X_test[['id', 'casual', 'registered', 'count']]
    X_test = X_test.drop(columns=target)
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/train.csv")
targets = ['casual', 'registered', 'count']
X_train, X_test, y_train, y_test = exam_data_load(df, target=targets)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8708, 10), (2178, 10), (8708, 4), (2178, 4))

-------

# EDA 및 전처리

In [37]:
X_train

,id,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
1550,1550,2011-04-10 11:00:00,2,0,0,2,16.40,20.455,87,8.9981
6454,6454,2012-03-06 04:00:00,1,0,1,1,7.38,8.335,51,19.9995
813,813,2011-02-17 08:00:00,1,0,1,1,13.12,16.665,57,6.0032
7069,7069,2012-04-12 22:00:00,2,0,1,1,17.22,21.210,35,12.9980
3217,3217,2011-08-03 23:00:00,3,0,1,2,27.88,31.820,89,6.0032
...,...,...,...,...,...,...,...,...,...,...
6384,6384,2012-03-03 06:00:00,1,0,0,2,14.76,18.180,93,6.0032
4720,4720,2011-11-09 18:00:00,4,0,1,1,18.04,21.970,67,7.0015
173,173,2011-01-08 12:00:00,1,0,0,2,8.20,9.090,59,23.9994
1244,1244,2011-03-16 16:00:00,1,0,1,2,16.40,20.455,76,11.0014


In [38]:
y_train

,id,casual,registered,count
1550,1550,93,109,202
6454,6454,0,3,3
813,813,18,267,285
7069,7069,15,143,158
3217,3217,4,52,56
...,...,...,...,...
6384,6384,1,6,7
4720,4720,21,387,408
173,173,8,90,98
1244,1244,30,121,151


In [39]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8708 entries, 1550 to 4989
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          8708 non-null   int64  
 1   datetime    8708 non-null   object 
 2   season      8708 non-null   int64  
 3   holiday     8708 non-null   int64  
 4   workingday  8708 non-null   int64  
 5   weather     8708 non-null   int64  
 6   temp        8708 non-null   float64
 7   atemp       8708 non-null   float64
 8   humidity    8708 non-null   int64  
 9   windspeed   8708 non-null   float64
dtypes: float64(3), int64(6), object(1)
memory usage: 748.3+ KB


---------



# datetime을 시계열 데이터로 변환

In [40]:
X_train['datetime'] = pd.to_datetime(X_train['datetime'])

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8708 entries, 1550 to 4989
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          8708 non-null   int64         
 1   datetime    8708 non-null   datetime64[ns]
 2   season      8708 non-null   int64         
 3   holiday     8708 non-null   int64         
 4   workingday  8708 non-null   int64         
 5   weather     8708 non-null   int64         
 6   temp        8708 non-null   float64       
 7   atemp       8708 non-null   float64       
 8   humidity    8708 non-null   int64         
 9   windspeed   8708 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 748.3 KB


------

# 계절, 주말 여부, 공휴일 여부 등이 이미 잘 작성되어 있는 데이터이므로 시간 데이터만 추출

In [42]:
X_train['hour'] = X_train['datetime'].dt.hour

In [43]:
X_train

,id,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,hour
1550,1550,2011-04-10 11:00:00,2,0,0,2,16.40,20.455,87,8.9981,11
6454,6454,2012-03-06 04:00:00,1,0,1,1,7.38,8.335,51,19.9995,4
813,813,2011-02-17 08:00:00,1,0,1,1,13.12,16.665,57,6.0032,8
7069,7069,2012-04-12 22:00:00,2,0,1,1,17.22,21.210,35,12.9980,22
3217,3217,2011-08-03 23:00:00,3,0,1,2,27.88,31.820,89,6.0032,23
...,...,...,...,...,...,...,...,...,...,...,...
6384,6384,2012-03-03 06:00:00,1,0,0,2,14.76,18.180,93,6.0032,6
4720,4720,2011-11-09 18:00:00,4,0,1,1,18.04,21.970,67,7.0015,18
173,173,2011-01-08 12:00:00,1,0,0,2,8.20,9.090,59,23.9994,12
1244,1244,2011-03-16 16:00:00,1,0,1,2,16.40,20.455,76,11.0014,16


In [44]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8708 entries, 1550 to 4989
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          8708 non-null   int64         
 1   datetime    8708 non-null   datetime64[ns]
 2   season      8708 non-null   int64         
 3   holiday     8708 non-null   int64         
 4   workingday  8708 non-null   int64         
 5   weather     8708 non-null   int64         
 6   temp        8708 non-null   float64       
 7   atemp       8708 non-null   float64       
 8   humidity    8708 non-null   int64         
 9   windspeed   8708 non-null   float64       
 10  hour        8708 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(7)
memory usage: 816.4 KB


----------

# X_test에 대해서도 동일하게 전처리 진행

In [45]:
X_test['datetime'] = pd.to_datetime(X_test['datetime'])
X_test['hour'] = X_test['datetime'].dt.hour

-------

# id 열 삭제

In [46]:
X_test_id = X_test['id']
X_train = X_train.drop(columns=['id', 'datetime'])
X_test = X_test.drop(columns=['id', 'datetime'])

------

# 모델링
# RandomForestRegressor 사용
casual과 registered 유저에 따라 수요가 다를 수 있으므로 따로 모델링 진행

In [47]:
from sklearn.ensemble import RandomForestRegressor
model_casual = RandomForestRegressor(random_state=2022)
model_casual.fit(X_train, y_train['casual'])
print("Train Data Casual Demand Score : ", model_casual.score(X_train, y_train['casual']))
print("Test Data Casual Demand Score : ", model_casual.score(X_test, y_test['casual']))
model_registered = RandomForestRegressor(random_state=2022)
model_registered.fit(X_train, y_train['registered'])
print("Train Data Registered Demand Score : ", model_registered.score(X_train, y_train['registered']))
print("Test Data Registered Demand Score : ", model_registered.score(X_test, y_test['registered']))

Train Data Casual Demand Score :  0.9803990828629902
Test Data Casual Demand Score :  0.8612829778856532
Train Data Registered Demand Score :  0.9766096638955819
Test Data Registered Demand Score :  0.8198367573684686


------

# 개별적으로 예측된 casual과 registered를 output 데이터에 입력하고 해당 데이터를 기반으로 전체 수요인 count 생성
# to_csv로 데이터 저장

In [48]:
prediction_casual = model_casual.predict(X_test)
prediction_registered = model_registered.predict(X_test)
output = pd.DataFrame({'id' : X_test_id, 'casual' : prediction_casual, 'registered' : prediction_registered})
output['count'] = output['casual'] + output['registered']

In [49]:
output

,id,casual,registered,count
6609,6609,58.970000,264.960,323.930000
6157,6157,3.520000,52.000,55.520000
4618,4618,89.390000,287.920,377.310000
613,613,4.930000,196.420,201.350000
10331,10331,8.690000,145.655,154.345000
...,...,...,...,...
1566,1566,0.260000,3.710,3.970000
4721,4721,24.220000,319.080,343.300000
5779,5779,3.116667,26.770,29.886667
7834,7834,50.335500,385.190,435.525500


In [50]:
output.to_csv("20220610.csv", index=False)

-------

# casual, registered, count 각각에 대해 RMSE 계산

In [51]:
from sklearn.metrics import mean_squared_error
mse_casual = mean_squared_error(y_test['casual'], output['casual'])
mse_registered = mean_squared_error(y_test['registered'], output['registered'])
mse_total = mean_squared_error(y_test['count'], output['count'])
print("casual RMSE : ", mse_casual**0.5)
print("registered RMSE : ", mse_registered**0.5)
print("total RMSE : ", mse_total**0.5)

casual RMSE :  18.686553147441302
registered RMSE :  62.834972313340494
total RMSE :  71.00070346779567
